In [1]:
import numpy as np
import pandas as pd
import os, shutil, glob
from PIL import Image
from itertools import islice

In [3]:
basedir = input("mnt/micro-images-pvc/PolinaG-U2OS") # /mnt/micro-images-pvc/PolinaG-U2OS


mnt/micro-images-pvc/PolinaG-U2OS /mnt/micro-images-pvc/PolinaG-U2OS


In [4]:
outputdir1 = input("/home/jovyan/scratch-shared/Linnea/Preprocessing/JoinChannels/outputdir1 ") # /scratch-shared/Linnea/Preprocessing/JoinChannels
#os.mkdir(outputdir1)

/home/jovyan/scratch-shared/Linnea/Preprocessing/JoinChannels/outputdir1  /home/jovyan/scratch-shared/Linnea/Preprocessing/JoinChannels/outputdir1


In [5]:
os.listdir(basedir)

['181212-U2OS-20X-BpA-HD-DB-high',
 '181212-U2OS-20X-BpA-HD-DB-low',
 '190124-U2OS-20X-CopyAP009069',
 '190131-U2OS-20X-CopyAP009064',
 '190131-U2OS-20X-CopyAP009065',
 '190131-U2OS-20X-CopyAP009066',
 '190131-U2OS-20X-CopyAP009067',
 '190131-U2OS-20X-CopyAP009068',
 '190201-U2OS-20X-CopyAP009063',
 '190201-U2OS-20X-CopyAP009083',
 '190206-U2OS-20X-CopyAP009070',
 '190206-U2OS-20X-CopyAP009071',
 '190206-U2OS-20X-CopyAP009072',
 '190207-U2OS-20X-CopyAP009073',
 '190207-U2OS-20X-CopyAP009074',
 '190207-U2OS-20X-CopyAP009075',
 '190207-U2OS-20X-CopyAP009076',
 '190207-U2OS-20X-CopyAP009077',
 '190207-U2OS-20X-CopyAP009078',
 '190207-U2OS-20X-CopyAP009079',
 '190208-U2OS-20X-CopyAP009080',
 '190208-U2OS-20X-CopyAP009081',
 '190208-U2OS-20X-CopyAP009082',
 '190208-U2OS-20X-CopyAP009084']

In [ ]:
# EXECUTE EITHER THIS CELL OR THE NEXT ONE
# window if crops is 4

def slide_window(img, dims=(270, 270)):
    window_height, window_width = dims
    y, x = img.shape[:2] #grab just height and width by taking the first two elements [:2]
    crop_images = np.zeros((crops,270,270,5)) # dtype="uint8"

    index = 0
    col = 0
    for i in range(y//window_height):
        row = 0
        for j in range(x//window_width):
            crop_images[index] = img[row:row+window_height, col:col+window_width, :]
            row += window_height
            index += 1
        col += window_width
    return crop_images

In [6]:
 # window if crops = 1

def slide_window(img, dims=(540, 540)):
    window_height, window_width = dims
    y, x = img.shape[:2]
    crop_images = np.zeros((crops,540,540,5)) # dtype="uint8"

    index = 0
    col = 0
    for i in range(y//window_height):
        row = 0
        for j in range(x//window_width):
            crop_images[index] = img[row:row+window_height, col:col+window_width, :]
            row += window_height
            index += 1
        col += window_width
    return crop_images

In [7]:
def anscombe(x):  # Makes dark images brighter. Needed???
    x = x.astype(np.float32)
    return (np.sqrt(x))

In [8]:
import datetime
now = datetime.datetime.now()
print ("Current date and time : ")
print (now.strftime("%Y-%m-%d %H:%M:%S"))

Current date and time : 
2020-02-24 12:04:37


In [9]:
def Normalization(x ,y, idx,crops):

    channels = x.shape[3]
    x = x.astype(np.float32)

    # For each channel, change range from (percentile 99.9 - percentile 1) to 255 - 0
             
    x[:,:,:,:] = x[:,:,:,:] * 16

    x = np.where(x < 255, x, 255)
    x = np.where(x > 0, x, 0)
    
    for i,j in zip(range(x.shape[0]), idx):
        xt = np.copy(x[i,:,:,:])
        #print ('image pixel intensity mean and std: ', xt.mean(), xt.std())
        imgs = slide_window(xt)
        for i,img in enumerate(imgs):
            imgMiSyHo = img[:,:,0:3]
            imgMiCoPh = img[:,:,[0,3,4]]
            
            img_cropped1 = Image.fromarray(imgMiSyHo.astype("uint8"))
            img_cropped2 = Image.fromarray(imgMiCoPh.astype("uint8"))
            
            # Save images to directories
            img_cropped1.save(outputdir1 + "/%s.png" % str(j+i))
            img_cropped2.save(outputdir2 + "/%s.png" % str(j+i))
    return

In [10]:
# read data file which links images to compound/concentration/therapeutic class etc,
data  = pd.read_csv('dataset.csv', low_memory=False, sep=';')
data.tail()

,Unnamed: 0,FileName_ORIG_CONCAVALIN,FileName_ORIG_HOECHST,FileName_ORIG_MITO,FileName_ORIG_PHALLOIDIN_WGA,FileName_ORIG_SYTO,Metadata_BARCODE,Metadata_SITE,Metadata_WELL,Nr,...,TREATMENT_h,cpd_num,library,cpd_structure,cpd_name,CBK_ID,batch_nr,therapeutic_class,therapeutic_effect,InChiKey
12295,12295,190208-U2OS-20X-CopyAP009084_G11_s5_w40F7812EB...,190208-U2OS-20X-CopyAP009084_G11_s5_w169B910AB...,190208-U2OS-20X-CopyAP009084_G11_s5_w314AF3809...,190208-U2OS-20X-CopyAP009084_G11_s5_w57207FDE4...,190208-U2OS-20X-CopyAP009084_G11_s5_w24FE693B0...,P009084,5,G11,21,...,48.0,378.0,Selleck-known kinase inhibitors,Cl.Cl.FC(F)(F)C1=CC2=C(C=C1)C(SCCCCCOC1=COC(CN...,EHT 1864,CBK293913C,BJ1856355,Rho,NaN,LSECOAJFCKFQJG-UHFFFAOYSA-N
12296,12296,190208-U2OS-20X-CopyAP009084_G11_s6_w4F220BDE7...,190208-U2OS-20X-CopyAP009084_G11_s6_w1998071EB...,190208-U2OS-20X-CopyAP009084_G11_s6_w3979B5E5F...,190208-U2OS-20X-CopyAP009084_G11_s6_w5FC0F153F...,190208-U2OS-20X-CopyAP009084_G11_s6_w270D170C9...,P009084,6,G11,21,...,48.0,378.0,Selleck-known kinase inhibitors,Cl.Cl.FC(F)(F)C1=CC2=C(C=C1)C(SCCCCCOC1=COC(CN...,EHT 1864,CBK293913C,BJ1856355,Rho,NaN,LSECOAJFCKFQJG-UHFFFAOYSA-N
12297,12297,190208-U2OS-20X-CopyAP009084_G11_s7_w4F8217623...,190208-U2OS-20X-CopyAP009084_G11_s7_w11380A259...,190208-U2OS-20X-CopyAP009084_G11_s7_w3ED603FAD...,190208-U2OS-20X-CopyAP009084_G11_s7_w544BC1D13...,190208-U2OS-20X-CopyAP009084_G11_s7_w2D94DE283...,P009084,7,G11,21,...,48.0,378.0,Selleck-known kinase inhibitors,Cl.Cl.FC(F)(F)C1=CC2=C(C=C1)C(SCCCCCOC1=COC(CN...,EHT 1864,CBK293913C,BJ1856355,Rho,NaN,LSECOAJFCKFQJG-UHFFFAOYSA-N
12298,12298,190208-U2OS-20X-CopyAP009084_G11_s8_w44EC3E75E...,190208-U2OS-20X-CopyAP009084_G11_s8_w1270DA46E...,190208-U2OS-20X-CopyAP009084_G11_s8_w3487FE656...,190208-U2OS-20X-CopyAP009084_G11_s8_w5725FA162...,190208-U2OS-20X-CopyAP009084_G11_s8_w2E96D33A7...,P009084,8,G11,21,...,48.0,378.0,Selleck-known kinase inhibitors,Cl.Cl.FC(F)(F)C1=CC2=C(C=C1)C(SCCCCCOC1=COC(CN...,EHT 1864,CBK293913C,BJ1856355,Rho,NaN,LSECOAJFCKFQJG-UHFFFAOYSA-N
12299,12299,190208-U2OS-20X-CopyAP009084_G11_s9_w4590CEADD...,190208-U2OS-20X-CopyAP009084_G11_s9_w157E8E9FA...,190208-U2OS-20X-CopyAP009084_G11_s9_w312165221...,190208-U2OS-20X-CopyAP009084_G11_s9_w58E8FA790...,190208-U2OS-20X-CopyAP009084_G11_s9_w2FDA9F990...,P009084,9,G11,21,...,48.0,378.0,Selleck-known kinase inhibitors,Cl.Cl.FC(F)(F)C1=CC2=C(C=C1)C(SCCCCCOC1=COC(CN...,EHT 1864,CBK293913C,BJ1856355,Rho,NaN,LSECOAJFCKFQJG-UHFFFAOYSA-N


In [11]:
labels = []
# keep track of images
count = 0
# keep track of dataset
dataset = 1
# List specifying images to be removed
rm_imgs = []

In [12]:
plates = []
for f in (f for f in os.listdir(basedir) if 'AP00' in f):
    plates.append(f[-7:])
plates = list(sorted(set(plates)))
print(plates)
#plates.remove('P009069')
plates.remove('P009084')
print(plates)
len(plates)

['P009063', 'P009064', 'P009065', 'P009066', 'P009067', 'P009068', 'P009069', 'P009070', 'P009071', 'P009072', 'P009073', 'P009074', 'P009075', 'P009076', 'P009077', 'P009078', 'P009079', 'P009080', 'P009081', 'P009082', 'P009083', 'P009084']
['P009063', 'P009064', 'P009065', 'P009066', 'P009067', 'P009068', 'P009069', 'P009070', 'P009071', 'P009072', 'P009073', 'P009074', 'P009075', 'P009076', 'P009077', 'P009078', 'P009079', 'P009080', 'P009081', 'P009082', 'P009083']


21

In [13]:
size = 540, 540 # Resize images that originally are 2160 x 2160
resampling = Image.LANCZOS # options: NEAREST, BILINEAR, BICUBIC, LANCZOS
crops = 1 # 1 or 4, or 16

In [ ]:
now = datetime.datetime.now()
print ("Current date and time : ")
print (now.strftime("%Y-%m-%d %H:%M:%S"))



# Iterate through directories, each containing one plate
for f in plates[:]:
    # Assign new variable for current plate
    plate_data = data[data['Metadata_BARCODE'].astype(str).str.contains(f)]
    
    idx = []
    plate_X = []
    plate_Y = []
    # Iterate through current plate
    for index, row in islice(plate_data.iterrows(), 0, None): 
        img_ORIG_CONCAVALIN   = Image.open(glob.glob(basedir + '/*/*/*/%s' %
                                                     row['FileName_ORIG_CONCAVALIN'])[0]).point(lambda i:i*(1./256)).convert('L').resize(size, resampling)
        img_ORIG_CONCAVALIN   = np.array(img_ORIG_CONCAVALIN)

        
        img_ORIG_HOECHST   = Image.open(glob.glob(basedir + '/*/*/*/%s' % 
                                                  row['FileName_ORIG_HOECHST'])[0]).point(lambda i:i*(1./256)).convert('L').resize(size, resampling)
        img_ORIG_HOECHST   = np.array(img_ORIG_HOECHST)
        
        img_ORIG_MITO   = Image.open(glob.glob(basedir + '/*/*/*/%s' %
                                               row['FileName_ORIG_MITO'])[0]).point(lambda i:i*(1./256)).convert('L').resize(size, resampling)
        img_ORIG_MITO   = np.array(img_ORIG_MITO)
        
        img_ORIG_PHALLOIDIN_WGA   = Image.open(glob.glob(basedir + '/*/*/*/%s' %
                                                         row['FileName_ORIG_PHALLOIDIN_WGA'])[0]).point(lambda i:i*(1./256)).convert('L').resize(size, resampling)
        img_ORIG_PHALLOIDIN_WGA   = np.array(img_ORIG_PHALLOIDIN_WGA)
        
        img_ORIG_SYTO   = Image.open(glob.glob(basedir + '/*/*/*/%s' %
                                               row['FileName_ORIG_SYTO'])[0]).point(lambda i:i*(1./256)).convert('L').resize(size, resampling)
        img_ORIG_SYTO   = np.array(img_ORIG_SYTO)
        
        img_stack   = np.dstack((img_ORIG_MITO, img_ORIG_SYTO, img_ORIG_HOECHST, img_ORIG_CONCAVALIN, img_ORIG_PHALLOIDIN_WGA)) 
        plate_X.append(img_stack)

        if row['WELL_ROLE']:# != 'control':
                [labels.append( [row['Metadata_BARCODE'],
                                row['Metadata_WELL'],
                                row['Metadata_SITE'],
                                row['WELL_ROLE'],
                                row['CMPD_ID'],
                                row['therapeutic_class'],
                                row['CMPD_CONC_uM']]) for i in range(crops)]

                idx.append(count)
                count += crops

        plate_Y.append([row['Metadata_BARCODE'],
                                row['Metadata_WELL'],
                                row['Metadata_SITE'],
                                row['CMPD_ID'],
                                row['CMPD_CONC_uM']])

            
    plate_Y      = np.asarray(plate_Y)
    dmso_idx     = np.where(plate_Y[:,3] == "DMSO")[0]
    non_dmso_idx = np.where(plate_Y[:,3] != "DMSO")[0]
    all_idx = np.where(plate_Y[:,3])[0]
    
    if len(non_dmso_idx) > 0:
        plate_X = np.asarray(plate_X)
        plate_X = anscombe(plate_X) ##########################################
        Normalization(plate_X[all_idx], plate_X[dmso_idx], idx, crops)
    print('Number of compounds transformed = ' + str(count) + '; dataset = ' + str(dataset))
    now = datetime.datetime.now()
    print ("Current date and time : ")
    print (now.strftime("%Y-%m-%d %H:%M:%S"))
    dataset += 1

df = pd.DataFrame(labels)
df.to_csv('labels.csv',sep=';')

now = datetime.datetime.now()
print ("Current date and time : ")
print (now.strftime("%Y-%m-%d %H:%M:%S"))

     

Current date and time : 
2020-02-24 12:04:54
